In [1]:
# imports
from transformers import pipeline, BertTokenizer, BertModel, BertConfig
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# https://github.com/jessevig/bertviz
from bertviz import head_view

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print(f'Length of BERT base vocabulary: {len(tokenizer.vocab)}')

Length of BERT base vocabulary: 30522


In [3]:
text = "A simple sentence!"

tokens = tokenizer.encode(text)
print(tokens)

[101, 1037, 3722, 6251, 999, 102]


In [4]:
# decode will re-construct the sentence with the added [CLS] and [SEP] token
tokenizer.decode(tokens)


'[CLS] a simple sentence! [SEP]'

In [5]:
text = "My friend told me about this class and I love it so far! She was right."

tokens = tokenizer.encode(text)
print(tokens)

[101, 2026, 2767, 2409, 2033, 2055, 2023, 2465, 1998, 1045, 2293, 2009, 2061, 2521, 999, 2016, 2001, 2157, 1012, 102]


In [6]:
# A nicer printout  of token ids and token strings

print(f'Text: {text}. Num tokens: {len(tokens)}')
for t in tokens:
    print(f'Token: {t}, subword: {tokenizer.decode([t])}')

Text: My friend told me about this class and I love it so far! She was right.. Num tokens: 20
Token: 101, subword: [CLS]
Token: 2026, subword: my
Token: 2767, subword: friend
Token: 2409, subword: told
Token: 2033, subword: me
Token: 2055, subword: about
Token: 2023, subword: this
Token: 2465, subword: class
Token: 1998, subword: and
Token: 1045, subword: i
Token: 2293, subword: love
Token: 2009, subword: it
Token: 2061, subword: so
Token: 2521, subword: far
Token: 999, subword: !
Token: 2016, subword: she
Token: 2001, subword: was
Token: 2157, subword: right
Token: 1012, subword: .
Token: 102, subword: [SEP]


In [8]:
# Let's load a vanilla BERT-base model. 
# Note we have to specify uncased because the vocab size / pre-trained vectors are different
model = BertModel.from_pretrained('bert-base-uncased')

In [10]:
"""
word embedding : context free word embedding
position embedding : embedding position
token type embedding : 0 or 1 used to lookup the segment embedding
"""

'\nword embedding : context free word embedding\nposition embedding : embedding position\ntoken type embedding : 0 or 1 used to lookup the segment embedding\n'

In [9]:
# Get all of the model's parameters as a list of tuples.
named_params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(named_params)))

print('==== Embedding Layer ====\n')
for p in named_params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in named_params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in named_params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (